# Práctica 1. Preprocesamiento y Tokenización de Texto

> **Nota:** Este notebook asume Python 3.8+ con las bibliotecas `nltk`, `transformers`, `emoji` y `tokenizers` instaladas.

In [ ]:
# Instalación de dependencias (ejecutar solo la primera vez)
!pip install -q nltk transformers emoji tokenizers

## 1. Introducción

El **preprocesamiento** es la primera etapa en cualquier *pipeline* de Procesamiento del Lenguaje Natural (PLN). Su objetivo es limpiar y estructurar el texto de entrada para facilitar la extracción de información útil.

### ¿Por qué es necesario?

Los modelos de PLN trabajan con representaciones numéricas del lenguaje. Por ejemplo, en un texto publicado en X, un texto sin limpiar o preprocesar suele presentar:

| Problema | Ejemplo | Consecuencia |
|---|---|---|
| URLs únicas | `https://t.co/abc123` | Inflan el vocabulario sin aportar significado |
| Menciones | `@usuario1`, `@usuario2` | Cada usuario es un token distinto |
| Mayúsculas inconsistentes | `Casa` vs `casa` | Se tratan como palabras diferentes. |
| Caracteres elongados | `buenoooo` | Dificultan la normalización. Cada elongación con longitud diferente se tratan como palabras diferentes |
| Emojis | 😍, 🔥 | Son información valiosa (emoción/valoración/evaluación) o ruido según la tarea |

En esta práctica trabajaremos con tuits en español — un corpus especialmente desafiante por su lenguaje informal, abreviaciones y mezcla de idiomas (*code-switching*).

### Pipeline típico de preprocesamiento

```
Texto crudo → Normalización → Tokenización → Filtrado → Representación vectorial
```

Cada paso reduce la dispersión del vocabulario y mejora la calidad de las representaciones.

## 2. Normalización de Texto

La **normalización** convierte el texto a una forma canónica. No existe una receta universal: las transformaciones dependen de la tarea (*clasificación de sentimientos* ≠ *reconocimiento de entidades*).

### Principio general
> *Cuanta más información eliminamos, más fácil resulta el aprendizaje automático pero mayor es el riesgo de perder señal relevante.*

Las transformaciones más habituales son:
- Conversión a minúsculas
- Eliminación/sustitución de URLs y menciones
- Tratamiento de emojis
- Reducción de elongaciones ("buenoooo" → "bueno")
- Eliminación de signos de puntuación redundantes

### 2.1. Expresiones Regulares para Usuarios y URLs

Las **expresiones regulares** (regex) son patrones que permiten identificar y transformar cadenas de texto de manera eficiente.

#### Anatomía de una regex útil en Twitter

| Patrón | Captura | Ejemplo |
|---|---|---|
| `@\w+` | Menciones | `@usuario`, `@NLP_Lab` |
| `https?://\S+` | URLs con protocolo | `https://t.co/abc` |
| `www\.\S+` | URLs sin protocolo | `www.ejemplo.com` |
| `#\w+` | Hashtags | `#MachineLearning` |
| `(.)\1{2,}` | Caracteres repetidos | `buenoooo` |

#### ¿Por qué normalizar con tokens especiales?

En lugar de eliminar `@usuario` directamente, se sustituye por `[USER]`. Esto:
1. **Preserva información estructural**: el modelo sabe que *alguien* fue mencionado.
2. **Reduce el vocabulario**: 10.000 usuarios distintos → 1 token.
3. **Facilita la anonimización** del corpus.

In [ ]:
import re

# ── Función de normalización básica ──────────────────────────────────
def normalizar_basico(texto):
    """Sustituye URLs y menciones por tokens especiales."""
    texto = re.sub(r'https?://\S+|www\.\S+', '[URL]', texto)  # URLs
    texto = re.sub(r'@\w+', '[USER]', texto)                    # Menciones
    return texto

def reducir_elongacion(texto):
    """'buenoooo' → 'buenoo' (conserva 2 para indicar énfasis)."""
    return re.sub(r'(.)\1{2,}', r'\1\1', texto)

def normalizar_hashtag(texto):
    """Extrae el texto de los hashtags eliminando el símbolo #."""
    return re.sub(r'#(\w+)', r'\1', texto)

# ── Batería de ejemplos ───────────────────────────────────────────────
ejemplos = [
    "Increíble charla de PLN en @universidad! Más info en https://t.co/nlp123",
    "Qué buenoooo el partido de @RealMadridCF 🔥 #LaLiga www.marca.com",
    "Hola @pepito y @maria, revisad http://docs.python.org pleaaaseee!!!",
]

print("=" * 65)
for ej in ejemplos:
    norm = normalizar_basico(ej)
    elon = reducir_elongacion(norm)
    print(f"Original  : {ej}")
    print(f"Norm.     : {norm}")
    print(f"Sin elong.: {elon}")
    print("-" * 65)

In [ ]:
# ── Comparativa cuantitativa ─────────────────────────────────────────
corpus_crudo = [
    "Visita https://openai.com y sigue a @OpenAI para novedades!",
    "El modelo de @Google está en https://cloud.google.com/ai",
    "Me ha encantado la presentación de @usuario1 en https://t.co/xyz",
    "Increíble!! @maria y @juan hablan de PLN en www.congreso-ia.es",
]

corpus_norm = [normalizar_basico(t) for t in corpus_crudo]

vocab_crudo = set(w for t in corpus_crudo for w in t.split())
vocab_norm  = set(w for t in corpus_norm  for w in t.split())

print(f"Tokens únicos ANTES de normalizar : {len(vocab_crudo)}")
print(f"Tokens únicos DESPUÉS de normalizar: {len(vocab_norm)}")
print(f"Reducción: {len(vocab_crudo) - len(vocab_norm)} tokens ({((len(vocab_crudo)-len(vocab_norm))/len(vocab_crudo))*100:.1f}%)")

### 2.2. Manejo de Emojis

Los emojis son **caracteres Unicode** que transmiten emoción, ironía y contexto con gran eficiencia. En Twitter, más del 25 % de los mensajes contienen al menos un emoji.

#### Estrategias de tratamiento

| Estrategia | Cuándo usarla | Resultado para 😍 |
|---|---|---|
| **Eliminar** | Tarea donde el sentimiento no importa (p. ej., extracción de entidades) | `""` |
| **Conservar** | Modelos modernos (BERT, GPT) que incluyen emojis en su vocabulario | `"😍"` |
| **Traducir a texto** | Modelos basados en palabras, análisis de sentimiento | `"cara sonriente con ojos de corazón"` |

#### La biblioteca `emoji`
Permite convertir emojis a su descripción textual oficial (en múltiples idiomas) o eliminarlos sistemáticamente.

In [ ]:
import emoji
import re

# ── 1. Eliminar emojis ────────────────────────────────────────────────
def eliminar_emojis(texto):
    return emoji.replace_emoji(texto, replace='')

# ── 2. Traducir emojis a texto ────────────────────────────────────────
def emojis_a_texto(texto, idioma='es'):
    return emoji.demojize(texto, language=idioma, delimiters=('[', ']'))

# ── 3. Extraer lista de emojis presentes ─────────────────────────────
def extraer_emojis(texto):
    return [c for c in texto if c in emoji.EMOJI_DATA]

# ── Ejemplos ──────────────────────────────────────────────────────────
tweets = [
    "Me encanta el PLN! 😍 Es lo mejor 🔥🔥🔥",
    "¡Victoria! 🏆🎉 @Seleccion gana el mundial",
    "Este bug me tiene 😤😤 ya no puedo más 💀",
]

print(f"{'Original':<45} | {'Sin emojis':<35} | {'Emojis encontrados'}")
print("-" * 110)
for t in tweets:
    sin = eliminar_emojis(t).strip()
    found = extraer_emojis(t)
    print(f"{t:<45} | {sin:<35} | {found}")

print()
print("── Traducción a texto ──")
for t in tweets:
    print(f"  {t}")
    print(f"  → {emojis_a_texto(t)}")
    print()

## 3. Tokenización: De la Palabra a la Subpalabra

La **tokenización** es el proceso de segmentar una cadena de texto en unidades mínimas de significado llamadas **tokens**. Es la piedra angular de cualquier sistema PLN: todo modelo recibe una secuencia de tokens como entrada.

### 3.1. Tokenización por Espacios (`str.split`)

El método más primitivo: divide el texto en cada espacio en blanco.

**Ventajas:** Rápido, sin dependencias, determinista.

**Problemas:**
- `"hola!"` ≠ `"hola"` → vocabulario inflado innecesariamente.
- No trata correctamente contracciones (`"don't"` → `["don't"]`).

In [ ]:
# ── Tokenización por split ───────────────────────────────────────────
ejemplos_split = [
    "El gato come. ¿El perro también come?",
    "Me encanta el PLN! 😍 #MachineLearning",
    "URL: https://t.co/abc y mención: @usuario",
]

print("── Tokenización con str.split() ──")
for texto in ejemplos_split:
    tokens = texto.split()
    print(f"  Texto  : {texto}")
    print(f"  Tokens : {tokens}")
    print(f"  Nº tok.: {len(tokens)}")
    print()

### 3.2. Tokenización Lingüística: NLTK TweetTokenizer

NLTK es una librería de Python para PLN con una gran cantidad de métodos que facilitan la programación de sistemas de PLN. Entre los disntitos métodos que ofrece están distintos tipos de *tokenizadores*. [Paquete de tokenización](https://www.nltk.org/api/nltk.tokenize.html), [Ejemplos](https://www.nltk.org/howto/tokenize.html).

NLTK ofrece un tokenizador **específico para redes sociales**, en particular para X, que aplica reglas lingüísticas para manejar correctamente:
- Signos de puntuación: los separa como tokens independientes.
- Hashtags: los conserva como unidad (`#MachineLearning` → 1 token).
- Emojis: los trata como tokens propios.
- Contracciones en inglés: `"don't"` → `["do", "n't"]`.

#### Parámetros clave de `TweetTokenizer`

| Parámetro | Por defecto | Efecto |
|---|---|---|
| `preserve_case` | `True` | Si `False`, convierte a minúsculas |
| `reduce_len` | `False` | Si `True`, reduce elongaciones ("buenoooo" → "buenoo") |
| `strip_handles` | `False` | Si `True`, elimina @menciones |

In [ ]:
import nltk
nltk.download('punkt', quiet=True)
nltk.download('punkt_tab', quiet=True)
from nltk.tokenize import TweetTokenizer, word_tokenize

tweet = "¡Hoooolaaaa! ¿Qué tal estás, señor García? Visita www.ejemplo.com o @escribeme 🎉"

tknzr_default  = TweetTokenizer()
tknzr_lower    = TweetTokenizer(preserve_case=False)
tknzr_reduce   = TweetTokenizer(preserve_case=False, reduce_len=True)
tknzr_nohandle = TweetTokenizer(preserve_case=False, reduce_len=True, strip_handles=True)

configs = [
    ("Default",               tknzr_default),
    ("lowercase",             tknzr_lower),
    ("lowercase + reduce_len",tknzr_reduce),
    ("+ strip_handles",       tknzr_nohandle),
]

print(f"Tweet original: {tweet}\n")
for nombre, tkn in configs:
    print(f"  [{nombre}]")
    print(f"    {tkn.tokenize(tweet)}")
    print()


print("── Comparativa split vs TweetTokenizer ──")
print(f"split          : {tweet.split()}")
print(f"TweetTokenizer : {TweetTokenizer().tokenize(tweet)}")
print(f"word_tokenize  : {word_tokenize(tweet, language='spanish')}")

### 3.3. Tokenización por Subpalabras

Los tokenizadores de **subpalabras** son el estándar de la industria desde 2018. En lugar de dividir por palabras completas o por caracteres individuales, aprenden un vocabulario de *fragmentos frecuentes* a partir de un corpus grande. El resultado es un equilibrio óptimo entre:

- **Cobertura:** toda palabra desconocida puede descomponerse en fragmentos conocidos → **cero `<UNK>`**.
- **Eficiencia:** el vocabulario es finito y manejable (10.000–256.000 entradas).
- **Morfología:** se capturan prefijos y sufijos recurrentes (`pre-`, `-ción`, `-mente`).

El más conocido es **BPE**, pero existen otros como **WordPiece**. Un poco más de *tokenizadores* a nivel de subpalabra en [HuggingFace](https://huggingface.co/docs/transformers/tokenizer_summary#byte-pair-encoding-bpe).

---

#### 3.3.1. Byte Pair Encoding (BPE)

**Idea central:** Construir el vocabulario fusionando iterativamente el par de símbolos más frecuente del corpus.

##### Algoritmo paso a paso

```
Corpus de entrenamiento (simplificado, 4 palabras):
  "bajo" × 5,  "baja" × 3,  "trabajo" × 4,  "trabaja" × 2

─── Inicialización ──────────────────────────────────────────────
Cada palabra se descompone en caracteres + marcador de fin </w>:
  b a j o         × 5
  b a j a         × 3
  t r a b a j o   × 4
  t r a b a j a   × 2

─── Iteración 1 ─────────────────────────────────────────────────
Par más frecuente: "a j"  (5+3+4+2 = 14 veces)
→ Fusionar:  b [aj] o,  b [aj] a,  t r a b [aj] o ...

─── Iteración 2 ─────────────────────────────────────────────────
Par más frecuente: "aj o"  (5+4 = 9 veces)
→ Fusionar: b [ajo],  t r a b [ajo] ...

─── Tras N iteraciones ──────────────────────────────────────────
Seguimos iterando hasta llegar al tamaño de vocabulario deseado.
```

##### Byte-level BPE: la variante moderna

Los modelos más recientes operan sobre **bytes UTF-8** en lugar de caracteres Unicode. Esto garantiza que *cualquier texto en cualquier idioma* puede tokenizarse sin caracteres desconocidos, ya que todo texto se puede representar con solo 256 símbolos base (un byte por símbolo).

| Modelo | Vocab | Año | Notas |
|---|---|---|---|
| **GPT-4o** | ~200 k | 2024 | `o200k_base`, optimizado para código y multilingüe |
| **LLaMA 3.1 / 3.2** | 128 k | 2024 | Gran mejora sobre LLaMA 2 (32 k), mejor cobertura multilingüe |
| **Mistral / Mixtral** | 32 k | 2023 | Compacto y eficiente para despliegue local |
| **Phi-4** | 100 k | 2024 | Heredado de tiktoken, foco en razonamiento |
| **DeepSeek-V3** | 128 k | 2024 | Entrenado masivamente en chino + inglés + código |
| **Qwen 2.5** | 152 k | 2024 | Vocabulario ampliado para chino y código |

---

#### 3.3.4. Resumen comparativo

| Propiedad | BPE | WordPiece |
|---|---|---|
| **Criterio de selección** | Frecuencia del par | PMI / verosimilitud |
| **Marcador de continuación** | Ninguno (`Ġ` en byte-level) | `##` al inicio del subword |
| **Modelos representativos (2024)** | GPT-4o, LLaMA 3, Phi-4 | ModernBERT, DeBERTa-v3 |

In [ ]:
from collections import Counter

def get_vocab(corpus: dict) -> dict:
    """Representa cada palabra como secuencia de caracteres + </w>."""
    return {' '.join(list(word)) + ' </w>': freq for word, freq in corpus.items()}

def get_pairs(vocab: dict) -> Counter:
    """Cuenta todos los pares de símbolos adyacentes."""
    pairs = Counter()
    for word, freq in vocab.items():
        symbols = word.split()
        for i in range(len(symbols) - 1):
            pairs[(symbols[i], symbols[i+1])] += freq
    return pairs

def merge_vocab(best_pair: tuple, vocab: dict) -> dict:
    """Fusiona el mejor par en todas las entradas del vocabulario."""
    bigram = ' '.join(best_pair)
    replacement = ''.join(best_pair)
    return {word.replace(bigram, replacement): freq for word, freq in vocab.items()}

# ── Corpus de ejemplo ────────────────────────────────────────────────
corpus = {'bajo': 5, 'baja': 3, 'trabajo': 4, 'trabaja': 2, 'caja': 6, 'caje': 1}

vocab = get_vocab(corpus)
print("Vocabulario inicial (nivel carácter):")
for w, f in vocab.items():
    print(f"  {f}× [{w}]")

# ── Ejecutar 8 iteraciones de BPE ────────────────────────────────────
N_MERGES = 8
merges = []

print(f"\n{'─'*60}\nEjecutando {N_MERGES} fusiones BPE...\n{'─'*60}")
for i in range(N_MERGES):
    pairs = get_pairs(vocab)
    if not pairs:
        break
    best_pair, freq = pairs.most_common(1)[0]
    vocab = merge_vocab(best_pair, vocab)
    merges.append(best_pair)
    merged = ''.join(best_pair)
    print(f"  Fusión {i+1:>2}: {best_pair[0]!r:8} + {best_pair[1]!r:8} → {merged!r:12} (frec. {freq})")

print(f"\nVocabulario FINAL:")
for w, f in sorted(vocab.items(), key=lambda x: -x[1]):
    tokens = w.replace(' </w>', '').split()
    print(f"  {f}× {tokens}")

In [ ]:
# ════════════════════════════════════════════════════════════════════
# Visualización: cómo segmenta cada tokenizador palabras difíciles
# ════════════════════════════════════════════════════════════════════
from transformers import AutoTokenizer

tok_bpe  = AutoTokenizer.from_pretrained("openai-community/gpt2")
tok_wordpiece = AutoTokenizer.from_pretrained("google-bert/bert-base-uncased")

palabras = [
    "tokenización",
    "chatgptear",
    "instagrameable",
    "microprocesador",
    "anticonstitucional",
    "COVID-19",
    "LLM",
    "fintuneado",          # "fine-tuned" hispanizado
    "embeddings",
    "multimodal",
]

print(f"  {'Palabra':<30} | {'BERT (WordPiece)':<55} | {'GPT-2 (Byte BPE)':<55}")
print("  " + "─" * 130)
for p in palabras:
    t_b = str(tok_wordpiece.tokenize(p))
    t_l = str(tok_bpe.tokenize(p))
    print(f"  {p:<30} | {t_b:<55} | {t_l:<55}")

Nota: actualmente ls *tokenizador* es un módulo muy importante, dado que cada modelo depende de un determinado *tokenizdor*.

## 4. Eliminación de Palabras Vacías (Stopwords)

Las **stopwords** (palabras vacías) son términos muy frecuentes que aportan poco valor semántico por sí solos: artículos, preposiciones, conjunciones, pronombres comunes...

### ¿Cuándo eliminar stopwords?

| Tarea | ¿Eliminar stopwords? | Razón |
|---|---|---|
| Clasificación de documentos | ✅ Sí (generalmente) | Reducen ruido y vocabulario |
| Modelos de bolsa de palabras (TF-IDF) | ✅ Sí | Los términos frecuentes dominan sin aportar discriminación |
| Modelos de lenguaje (BERT, GPT) | ❌ No | Estos modelos aprenden contexto completo |
| Traducción automática | ❌ No | Gramática completa necesaria |
| Análisis de sentimientos | ⚠️ Cuidado | "no me gusta" → si se elimina "no", invierte el sentimiento |

### ⚠️ Precaución con la negación

Eliminar stopwords puede **destruir la semántica negativa**:

```
"no me gusta"  → eliminar 'no' → "me gusta"  ← ¡Sentimiento invertido!
```

In [ ]:
import nltk
from nltk.corpus import stopwords
from collections import Counter
nltk.download('stopwords', quiet=True)

stop_es = set(stopwords.words('spanish'))
print(f"Stopwords en español (NLTK): {len(stop_es)} términos")
print(f"Muestra: {sorted(list(stop_es))[:20]}")

In [ ]:
from nltk.tokenize import TweetTokenizer

tknzr = TweetTokenizer(preserve_case=False)
stop_es = set(stopwords.words('spanish'))

tweets_ejemplo = [
    "El aprendizaje automático es el futuro de la inteligencia artificial",
    "no me gusta nada esta película, es muy aburrida",
    "España gana el partido gracias al gol en el último minuto",
]

print("── Efecto de eliminar stopwords ──\n")
for tweet in tweets_ejemplo:
    tokens   = tknzr.tokenize(tweet)
    sin_stop = [t for t in tokens if t not in stop_es]
    print(f"  Original   : {tweet}")
    print(f"  Todos      : {tokens}")
    print(f"  Sin stop.  : {sin_stop}")
    if 'no' in tokens and 'no' not in sin_stop:
        print(f"  ⚠️  Se eliminó 'no' — posible inversión semántica")
    print()

In [ ]:
# ── Stopwords personalizadas ──────────────────────────────────────────
stop_custom = stop_es.copy()
stop_custom.update(['rt', 'vía', 'cc', '[user]', '[url]', 'q', 'x', 'xq'])
palabras_a_conservar = {'no', 'ni', 'nunca', 'jamás', 'tampoco', 'sin'}
stop_custom -= palabras_a_conservar

print(f"Stopwords personalizadas: {len(stop_custom)}")
print(f"Añadidas: {stop_custom - stop_es}")
print(f"Preservadas (eliminadas del filtro): {palabras_a_conservar}")

tweet_neg = "no me gusta nada este producto, nunca lo recomendaría"
tokens_neg      = tknzr.tokenize(tweet_neg)
filtrado_nltk   = [t for t in tokens_neg if t not in stop_es]
filtrado_custom = [t for t in tokens_neg if t not in stop_custom]
print(f"\nTweet: {tweet_neg}")
print(f"Stop NLTK  : {filtrado_nltk}")
print(f"Stop custom: {filtrado_custom}  ← preserva 'no' y 'nunca'")

In [ ]:
# ── Análisis de frecuencias: con vs sin stopwords ─────────────────────
corpus = [
    "El gato come pescado todos los días",
    "El perro come carne y huesos",
    "El gato y el perro juegan en el jardín",
    "Los animales domésticos son compañía para las personas",
    "El pájaro canta en el árbol del jardín",
]

todos_tokens = [t for frase in corpus for t in tknzr.tokenize(frase)]
sin_stop_tok = [t for t in todos_tokens if t not in stop_es]

freq_con = Counter(todos_tokens)
freq_sin = Counter(sin_stop_tok)

print("── Top 10 tokens CON stopwords ──")
for token, n in freq_con.most_common(10):
    print(f"  {token:<15} {'█' * n} ({n})")

print()
print("── Top 10 tokens SIN stopwords ──")
for token, n in freq_sin.most_common(10):
    print(f"  {token:<15} {'█' * n} ({n})")

##5 *Tokenizción* en oraciones o segmentación

Aunque está práctica se está centrando en textos procedentes de plataformas de microblogging como X, es posible procesar textos/documntos más laragos, para lo cual es interesante segmentar o identificar las oraciones de dicho texto.

NLTK nos ofrece la posiblidad de segmentar en oraciones. Veamos un ejemplo

In [ ]:
from nltk.tokenize import sent_tokenize


texto = '''En un lugar de la Mancha, de cuyo nombre no quiero acordarme, no ha
mucho tiempo que vivía un hidalgo de los de lanza en astillero, adarga antigua,
rocín flaco y galgo corredor. Una olla de algo más vaca que carnero, salpicón
las más noches, duelos y quebrantos los sábados, lantejas los viernes, algún
palomino de añadidura los domingos, consumían las tres partes de su hacienda.
El resto della concluían sayo de velarte, calzas de velludo para las fiestas,
con sus pantuflos de lo mismo, y los días de entresemana se honraba con su
vellorí de lo más fino. Tenía en su casa una ama que pasaba de los cuarenta,
y una sobrina que no llegaba a los veinte, y un mozo de campo y plaza, que así
ensillaba el rocín como tomaba la podadera. Frisaba la edad de nuestro hidalgo
con los cincuenta años; era de complexión recia, seco de carnes, enjuto de
rostro, gran madrugador y amigo de la caza.'''

sentences = sent_tokenize(texto)
sentences = [s.replace('\n','') for s in sentences]
print(f"Número de oraciones: {len(sentences)}")
for i in range(len(sentences)):
  print(f"Oración {i+1}: {sentences[i]}")
print(sentences)


## 6. Pipeline Completo de Preprocesamiento

Combinamos todos los pasos anteriores en una función reutilizable y parametrizable según la tarea.

In [ ]:
import re, emoji, nltk
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords
nltk.download('stopwords', quiet=True)

def pipeline_preprocesamiento(
    texto,
    normalizar_urls=True,
    normalizar_users=True,
    normalizar_hashtags=False,
    tratar_emojis='conservar',   # 'conservar' | 'eliminar' | 'texto'
    lowercase=True,
    reduce_elongaciones=True,
    eliminar_stopwords=False,
    stopwords_extra=None,
):
    if normalizar_urls:
        texto = re.sub(r'https?://\S+|www\.\S+', '[URL]', texto)
    if normalizar_users:
        texto = re.sub(r'@\w+', '[USER]', texto)
    if normalizar_hashtags:
        texto = re.sub(r'#(\w+)', r'\1', texto)
    if tratar_emojis == 'eliminar':
        texto = emoji.replace_emoji(texto, replace='')
    elif tratar_emojis == 'texto':
        texto = emoji.demojize(texto, language='es', delimiters=(' [', '] '))
    if reduce_elongaciones:
        texto = re.sub(r'(.)\1{2,}', r'\1\1', texto)
    tknzr = TweetTokenizer(preserve_case=(not lowercase), reduce_len=reduce_elongaciones)
    tokens = tknzr.tokenize(texto)
    if eliminar_stopwords:
        stop = set(stopwords.words('spanish'))
        if stopwords_extra:
            stop.update(stopwords_extra)
        tokens = [t for t in tokens if t not in stop]
    return tokens

# ── Prueba con distintas configuraciones ─────────────────────────────
tweet_test = "Increíble!! 😍😍 @usuario dice que https://t.co/abc es lo mejorrr #PLN"

configs = {
    "Mínimo"       : dict(eliminar_stopwords=False, tratar_emojis='conservar'),
    "Clasificación": dict(eliminar_stopwords=True,  tratar_emojis='eliminar',  normalizar_hashtags=True),
    "Sentimiento"  : dict(eliminar_stopwords=False, tratar_emojis='texto',     normalizar_hashtags=True),
}
print(f"Tweet: {tweet_test}\n")
for nombre, kwargs in configs.items():
    tokens = pipeline_preprocesamiento(tweet_test, **kwargs)
    print(f"  [{nombre}]\n    {tokens}\n")

---
# EJERCICIOS

Para estos ejercicios, utilizaremos un conjunto de tweets en español (`cost.csv`, disponible en el material complementario). El objetivo es comparar cómo afectan las distintas técnicas de tokenización y limpieza a las estadísticas del corpus.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

#A partir de aquí, a trabajar...

#### Ejercicio 1: Limpieza y Anonimización

Crea una función llamada `preprocesar()` que reciba un tweet y:

1. Sustituya todas las URLs por `[URL]`.
2. Sustituya todas las menciones por `[USER]`.
3. Convierta todo el texto a minúsculas.
4. **Pregunta:** ¿Cuántos tokens únicos de `[USER]` y `[URL]` hay en el corpus tras procesarlo?

#### Ejercicio 2: Comparativa de Vocabulario

Aplica tres tipos de tokenización al corpus limpio (Split, NLTK TweetTokenizer y WordPiece/BPE) y calcula lo siguiente para cada uno:
1. Tokens totales
2. Tamaño del vocabulario (tokens únicos)

Analiza los resultados obtenidos. ¿Qué hace que el tamaño del vocabulario sea mayor o menor en función del tokenizador escogido?

#### Ejercicio 3: Stopwords y Frecuencia

Utilizando el tokenizador de NLTK:

1. Muestra los 10 tokens más frecuentes **con** stopwords.
2. Muestra los 10 tokens más frecuentes **sin** stopwords.
3. Muestra los 10 tokens más frecuentes **sin** caracteres no alfanuméricos.

¿Qué diferencia observas en la capacidad de estos tokens para describir el "tema" o temática de los tweets?